In [9]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from preprocess import pos_tagger
from preprocess import words_lemmatizer
from preprocess import clean_by_freq, clean_by_len, clean_by_stopwords
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\parkf\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv('imdb.tsv', delimiter = "\\t")

C:\Users\parkf\AppData\Local\Temp\ipykernel_18040\756002784.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('imdb.tsv', delimiter = "\\t")


In [3]:
# 소문자로 정규화
df['review'] = df['review'].str.lower()

In [4]:
# 문장 토큰화
df['sent_tokens'] = df['review'].apply(sent_tokenize)
print(df['sent_tokens'][0])

['"watching time chasers, it obvious that it was made by a bunch of friends.', 'maybe they were sitting around one day in film school and said, \\""hey, let\'s pool our money together and make a really bad movie!\\"" or something like that.', 'what ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc.', "all corners were cut, except the one that would have prevented this film's release.", 'life\'s like that."']


In [5]:
# 품사 태깅
df['pos_tagged_tokens'] = df['sent_tokens'].apply(pos_tagger)
print(df['pos_tagged_tokens'][0])

[('``', '``'), ('watching', 'JJ'), ('time', 'NN'), ('chasers', 'NNS'), (',', ','), ('it', 'PRP'), ('obvious', 'VBZ'), ('that', 'IN'), ('it', 'PRP'), ('was', 'VBD'), ('made', 'VBN'), ('by', 'IN'), ('a', 'DT'), ('bunch', 'NN'), ('of', 'IN'), ('friends', 'NNS'), ('.', '.'), ('maybe', 'RB'), ('they', 'PRP'), ('were', 'VBD'), ('sitting', 'VBG'), ('around', 'IN'), ('one', 'CD'), ('day', 'NN'), ('in', 'IN'), ('film', 'NN'), ('school', 'NN'), ('and', 'CC'), ('said', 'VBD'), (',', ','), ('\\', 'FW'), ("''", "''"), ("''", "''"), ('hey', 'NN'), (',', ','), ('let', 'VB'), ("'s", 'POS'), ('pool', 'VB'), ('our', 'PRP$'), ('money', 'NN'), ('together', 'RB'), ('and', 'CC'), ('make', 'VB'), ('a', 'DT'), ('really', 'RB'), ('bad', 'JJ'), ('movie', 'NN'), ('!', '.'), ('\\', 'NN'), ("''", "''"), ("''", "''"), ('or', 'CC'), ('something', 'NN'), ('like', 'IN'), ('that', 'DT'), ('.', '.'), ('what', 'WP'), ('ever', 'RB'), ('they', 'PRP'), ('said', 'VBD'), (',', ','), ('they', 'PRP'), ('still', 'RB'), ('ended',

In [6]:
# 표제어 추출
df['lemmatized_tokens'] = df['pos_tagged_tokens'].apply(words_lemmatizer)
print(df['lemmatized_tokens'][0])

['``', 'watching', 'time', 'chaser', ',', 'it', 'obvious', 'that', 'it', 'be', 'make', 'by', 'a', 'bunch', 'of', 'friend', '.', 'maybe', 'they', 'be', 'sit', 'around', 'one', 'day', 'in', 'film', 'school', 'and', 'say', ',', '\\', "''", "''", 'hey', ',', 'let', "'s", 'pool', 'our', 'money', 'together', 'and', 'make', 'a', 'really', 'bad', 'movie', '!', '\\', "''", "''", 'or', 'something', 'like', 'that', '.', 'what', 'ever', 'they', 'say', ',', 'they', 'still', 'end', 'up', 'make', 'a', 'really', 'bad', 'movie', '--', 'dull', 'story', ',', 'bad', 'script', ',', 'lame', 'acting', ',', 'poor', 'cinematography', ',', 'bottom', 'of', 'the', 'barrel', 'stock', 'music', ',', 'etc', '.', 'all', 'corner', 'be', 'cut', ',', 'except', 'the', 'one', 'that', 'would', 'have', 'prevent', 'this', 'film', "'s", 'release', '.', 'life', "'s", 'like', 'that', '.', "''"]


In [7]:
# 추가 전처리 (빈도 1 이하, 단어 길이 2 이하 단어 제거, 불용어 제거)
stopwords_set = set(stopwords.words('english'))

df['cleaned_tokens'] = df['lemmatized_tokens'].apply(lambda x: clean_by_freq(x, 1))
df['cleaned_tokens'] = df['cleaned_tokens'].apply(lambda x: clean_by_len(x, 2))
df['cleaned_tokens'] = df['cleaned_tokens'].apply(lambda x: clean_by_stopwords(x, stopwords_set))

df[['cleaned_tokens']]

,cleaned_tokens
0,"[make, one, film, say, make, really, bad, movi..."
1,"[film, film]"
2,"[new, york, joan, barnard, elvire, audrey, bar..."
3,"[film, film, jump, send, n't, jump, radio, n't..."
4,"[site, movie, bad, even, movie, movie, make, m..."
5,"[ehle, northam, wonderful, wonderful, ehle, no..."
6,"[role, movie, n't, author, book, funny, author..."
7,"[plane, ceo, search, rescue, mission, call, ce..."
8,"[gritty, movie, movie, keep, sci-fi, good, kee..."
9,"[girl, girl]"


In [8]:
# 정수 인코딩

In [10]:
tokens = df['cleaned_tokens'][4]

vocab = Counter(tokens)
vocab = vocab.most_common()

print(vocab)

[('movie', 10), ('jim', 7), ('stand-up', 3), ('day', 3), ('really', 3), ('terrible', 3), ('site', 2), ('bad', 2), ('even', 2), ('make', 2), ('special', 2), ('describe', 2), ('like', 2), ('actor', 2), ('love', 2), ('stand', 2), ('comedian', 2)]


In [11]:
# 각 단어에 인덱스 부여
word_to_idx = {}
i = 0

for (word, frequency) in vocab:
    i = i + 1
    word_to_idx[word] = i

print(word_to_idx)

{'movie': 1, 'jim': 2, 'stand-up': 3, 'day': 4, 'really': 5, 'terrible': 6, 'site': 7, 'bad': 8, 'even': 9, 'make': 10, 'special': 11, 'describe': 12, 'like': 13, 'actor': 14, 'love': 15, 'stand': 16, 'comedian': 17}


In [12]:
# 토큰들을 부여된 인덱스로 변경
encoded_idx = []

for token in tokens:
    idx = word_to_idx[token]
    encoded_idx.append(idx)

print(encoded_idx)

[7, 1, 8, 9, 1, 1, 10, 1, 11, 12, 1, 1, 12, 1, 2, 10, 3, 4, 3, 2, 13, 2, 14, 15, 16, 4, 17, 11, 2, 4, 9, 7, 15, 2, 3, 2, 14, 1, 16, 17, 2, 13, 5, 6, 5, 6, 1, 6, 5, 8, 1]


In [13]:
# 전체 코퍼스의 토큰들을 전부 합하여 단어의 등장 빈도를 계산
tokens = sum(df['cleaned_tokens'], [])

print(tokens)

['make', 'one', 'film', 'say', 'make', 'really', 'bad', 'movie', 'like', 'say', 'make', 'really', 'bad', 'movie', 'bad', 'one', 'film', 'like', 'film', 'film', 'new', 'york', 'joan', 'barnard', 'elvire', 'audrey', 'barnard', 'john', 'saxon', 'italy', 'etruscan', 'tomb', 'joan', 'italy', 'colleague', 'italy', 'maggot', 'maggot', 'joan', 'drug', 'drug', 'tomb', 'colleague', 'story', 'end', 'new', 'york', 'joan', 'colleague', 'romantic', 'end', 'waste', 'time', 'watch', 'story', 'romantic', 'end', 'elvire', 'audrey', 'john', 'saxon', 'maggot', 'watch', 'etrusco', 'watch', 'waste', 'time', 'etrusco', 'etruscan', 'film', 'film', 'jump', 'send', "n't", 'jump', 'radio', "n't", 'send', 'reporter', 'fear', 'jump', 'fear', 'radio', 'reporter', "n't", 'radio', "n't", "n't", 'site', 'movie', 'bad', 'even', 'movie', 'movie', 'make', 'movie', 'special', 'describe', 'movie', 'movie', 'describe', 'movie', 'jim', 'make', 'stand-up', 'day', 'stand-up', 'jim', 'like', 'jim', 'actor', 'love', 'stand', 'da

In [14]:
# 합쳐진 토큰 리스트로 빈도 계산, 많이 등장한 순으로 정렬하여 정수 인덱스 부여
word_to_idx = {}
i = 0
tokens = sum(df['cleaned_tokens'], [])

vocab = Counter(tokens)
vocab = vocab.most_common()

for (word, frequency) in vocab:
    i = i + 1
    word_to_idx[word] = i

In [15]:
# 결과 확인
print(word_to_idx)

{'movie': 1, 'film': 2, "n't": 3, 'scene': 4, 'bad': 5, 'time': 6, 'reason': 7, 'make': 8, 'jim': 9, 'good': 10, 'one': 11, 'like': 12, 'could': 13, "'re": 14, 'quastel': 15, 'really': 16, 'even': 17, 'monster': 18, 'joan': 19, 'love': 20, 'author': 21, 'try': 22, 'dialogue': 23, 'idea': 24, 'italy': 25, 'colleague': 26, 'maggot': 27, 'end': 28, 'watch': 29, 'jump': 30, 'radio': 31, 'stand-up': 32, 'day': 33, 'terrible': 34, 'ehle': 35, 'northam': 36, 'search': 37, 'rescue': 38, 'call': 39, 'knowles': 40, 'henriksen': 41, 'easily': 42, 'see': 43, 'appear': 44, 'get': 45, 'character': 46, 'think': 47, 'use': 48, 'whether': 49, 'need': 50, 'though': 51, 'sci-fi': 52, 'look': 53, 'say': 54, 'new': 55, 'york': 56, 'barnard': 57, 'elvire': 58, 'audrey': 59, 'john': 60, 'saxon': 61, 'etruscan': 62, 'tomb': 63, 'drug': 64, 'story': 65, 'romantic': 66, 'waste': 67, 'etrusco': 68, 'send': 69, 'reporter': 70, 'fear': 71, 'site': 72, 'special': 73, 'describe': 74, 'actor': 75, 'stand': 76, 'comed

In [16]:
# 데이터 프레임의 각 로우에 있는 토큰들을 정수 인코딩
def idx_encoder(tokens, word_to_idx):
    encoded_idx = []
    
    for token in tokens:
        idx = word_to_idx[token]
        encoded_idx.append(idx)
        
    return encoded_idx

df['integer_encoded'] = df['cleaned_tokens'].apply(lambda x: idx_encoder(x, word_to_idx))
df[['integer_encoded']]

,integer_encoded
0,"[8, 11, 2, 54, 8, 16, 5, 1, 12, 54, 8, 16, 5, ..."
1,"[2, 2]"
2,"[55, 56, 19, 57, 58, 59, 57, 60, 61, 25, 62, 6..."
3,"[2, 2, 30, 69, 3, 30, 31, 3, 69, 70, 71, 30, 7..."
4,"[72, 1, 5, 17, 1, 1, 8, 1, 73, 74, 1, 1, 74, 1..."
5,"[35, 36, 78, 78, 35, 36, 79, 79, 35, 36]"
6,"[80, 1, 3, 21, 81, 82, 21, 21, 80, 3, 82, 83, ..."
7,"[85, 86, 37, 38, 87, 39, 86, 88, 40, 89, 41, 9..."
8,"[120, 1, 1, 121, 52, 10, 121, 122, 53, 1, 52, ..."
9,"[123, 123]"
